In [1]:
from ase.io import read
from ase import Atoms
from ase.optimize import *
from ase.visualize import view
from ase.md import *
from ase.calculators.mopac import *

import nglview

In [2]:
#
# In this tutoiral notebook, you will optimize a single water molecule from different starting conformations,
# now using a semi-empirical force-field and the BFGS Quasi-Newton method.
#

#
# In 01.01_newton_on_harmonic_potential_of_water.ipynb,
# you saw that from all three starting conformations, the Newton method converged to the minimum structure within 
# 1 step because we had used a simple, 2D-harmonic potential for water.
# For semi-empirical force fields, however, which also include non-bonded interactions, 
# the "linear" H2O conformation represents a saddle point, on which the Newton optimizer would get "stuck" 
# and not converge to the minimum, whereas a Quasi-Newton method indeed finds the minimum energy structure.
#
# Unfortunately, the Newton optimizer is not implemented in ASE.
# However, we can already sense this saddle-point behavior of the "linear" water conformation by
# using the BFGS Quasi-Newton optimizer and comparing differences in how fast the optimzation convergerges 
# to the minimum structure.
#
# TASK: 
# (i)  Optimize the three different water conformations below, with varying optimization iterations
#      (e.g. numiterations=2, 5, 10)
# (ii) Comparing the initial and optimized structures through visualization
#      (via the nglview module below, or VMD, Pymol, ...),
#      can you confirm that the linear water conformation takes longer to converge to the minimum structure?
#

#model = "h2o"
#model = "h2o_90"
model = "h2o_linear"

# number of optimzation iterations
numiterations = 20

infile  = "input/%s.pdb" % model
water   = read(infile, format="pdb")
molecule = Atoms(water)

# visualize initial structure
wdg_init = nglview.show_ase(molecule)
wdg_init.add_representation('ball+stick')
wdg_init.center_view(range(3))

In [3]:
# instantiate Mopac
calc = Mopac(restart=0, spin=0, OPT=False, functional='PM6', job_type='NOANCI 1SCF GRADIENTS AUX(0,PRECISION=9)', RELSCF=0.0001)

#
# Compare the initial PDB files before and after Optimization - what do you observe?
# What is your conclusion with respect to what you know from 01_water_ff.ipynb ?
#

molecule.set_calculator(calc)

print "model", model
ener = molecule.get_potential_energy()
print "potential energy:", ener
grad = molecule.get_forces()
print "gradient", grad

dyn = QuasiNewton(molecule, trajectory = "output/" + model + '.water.QN_opt.traj')
dyn.run(fmax=0.005, steps = numiterations)
outfile = "output/" + model + ".QN_opt.pdb"
molecule.write(outfile)

model h2o_linear
potential energy: -1.40506799782
gradient [[ -3.36939171e-04  -3.22542285e-04   2.85778188e-03]
 [  6.38883501e-04   1.81439792e-03   3.17160425e+00]
 [  6.22496202e-03   4.25835606e-03  -3.15889072e+00]]
BFGSLineSearch:   0[  0]  19:05:55       -1.405068       3.1716
BFGSLineSearch:   1[  1]  19:05:55       -1.995258       1.2994
BFGSLineSearch:   2[  3]  19:05:55       -2.032130       0.0353
BFGSLineSearch:   3[  5]  19:05:56       -2.032146       0.0056
BFGSLineSearch:   4[ 78]  19:06:02       -2.032147       0.0115
BFGSLineSearch:   5[ 87]  19:06:02       -2.134316       1.8143
BFGSLineSearch:   6[ 91]  19:06:03       -2.134327       1.8220
BFGSLineSearch:   7[ 94]  19:06:03       -2.297197       1.2052
BFGSLineSearch:   8[ 95]  19:06:04       -2.353635       0.2349
BFGSLineSearch:   9[ 96]  19:06:04       -2.354767       0.0706
BFGSLineSearch:  10[ 97]  19:06:04       -2.354934       0.0197
BFGSLineSearch:  11[ 98]  19:06:04       -2.354945       0.0084
BFGSLineSe

In [9]:
# visualize initial structure
wdg_init.display(gui=True)

ValueError: None is not in list

ValueError: None is not in list

ValueError: None is not in list

In [10]:
# visualize optimized structure
wdg = nglview.show_ase(molecule)
wdg.add_representation('ball+stick')
wdg.center_view(range(3))
wdg.display(gui=True)